In [65]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<h3>This munging uses data scraped 3/25/2017</h3>

The Department of Human Services provided us with seven spreadsheets that each have information about substantiated complaints against assisted living, residential care, and nursing facilities in Oregon. The purpose of this notebook is to mung them, standardizing the column names, removing unnecessary columns, and cleaning some fields. The second purpose of this notebook is to get initial ownership date for facilities from the owner_history table and assign it to each facility.

<h1>Complaints mung</h1>

<h3>Start with the 10-year data that does not have narratives.</h3>

<h4>Import, clean, concat.</h4>

In [66]:
#Five years of detailed complaint data for all four kinds of facilities (Residential Care, Assisted Living, Nursing, and Adult Foster Home)
detailed = pd.read_excel('../../data/raw/Oregonian Abuse records 5 years May 2016.xlsx', header=3)
#Ten years of non-detailed complaints for Nursing Facilities
NF_complaints = pd.read_excel('../../data/raw/Copy of Oregonian Data Request Facility Abuse Records April 2016   Reviewed.xlsx',sheetname='NF Complaints')
#Ten years of non-detailed complaints for Assisted Living Facilities
ALF_complaints = pd.read_excel('../../data/raw/Copy of Oregonian Data Request Facility Abuse Records April 2016   Reviewed.xlsx',sheetname='ALF Complaints')
#Ten years of non-detailed complaints for Residential Care Facilities
RCF_complaints = pd.read_excel('../../data/raw/Copy of Oregonian Data Request Facility Abuse Records April 2016   Reviewed.xlsx',sheetname='RCF Complaints')

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [67]:
#NF has an inconsistently named column
NF_complaints.rename(columns={'Abuse_CbcAbuse': 'CbcAbuse'}, inplace=True)

In [68]:
ten_year_complaints = pd.concat([RCF_complaints,ALF_complaints,NF_complaints], ignore_index=True).reset_index().drop('index',1)

In [69]:
ten_year_complaints.rename(columns={'Abuse_Number':'abuse_number', 'Facility ID':'facility_id','Incident Date':'incident_date','Fac Type': 'facility_type',
'Investigation Results':'results_1','FacilityInvestResultsAbuse':'results_2','FacilityInvestResultsRule':'results_3','OutcomeCode':'outcome_code',
'CbcAbuse':'abuse_type'}, inplace=True)

In [70]:
ten_year_complaints = ten_year_complaints[['abuse_number','facility_id','incident_date','results_1',
                                           'results_2','results_3','outcome_code','abuse_type']][ten_year_complaints['abuse_number'].notnull()]

<h4>There are 52 complaints that have been mislabelled as unsubstantiated.</h4>

In [71]:
sub_comps = pd.read_excel('../../data/raw/52 mislabelled as unsubstantiated.xlsx', header=None, names=['abuse_number'])

In [72]:
miss_comps = sub_comps.merge(ten_year_complaints, how = 'left', left_on='abuse_number',right_on='abuse_number')#.count()

This dataset contains unsubstantiated complaints, which we don't need. There are three columns that indicate substantiation. A DHS person explained that if any one of them has the word 'substantiated,' then the complaint was substantiated. 

In [73]:
ten_year_complaints = ten_year_complaints[(ten_year_complaints['results_1']=='Substantiated')|
                   (ten_year_complaints['results_2']=='Substantiated')|
                   (ten_year_complaints['results_3']=='Substantiated')]

In [74]:
ten_year_complaints = pd.concat([ten_year_complaints,miss_comps]).reset_index().drop('index',1)

In [75]:
ten_year_ready = ten_year_complaints[['abuse_number','facility_id','incident_date','outcome_code','abuse_type']].reset_index().drop('index',1)

<h3>Now we prepare the five-year, detailed data</h3>

The 'detailed' data is a five-year set of substantiated complaints against all facilities, including adult foster homes, which we don't want.

In [76]:
detailed.rename(columns={'Abuse_Number':'abuse_number','Facility ID':'facility_id',
                'Incident Date':'incident_date','Investigation Results':'results_1',
                'Facility Invest Results Abuse':'results_2','Facility Invest Results Rule':'results_3',
               'Outcome Code':'outcome_code','Action Notes':'action_notes','Outcome Notes':'outcome_notes',
               'Cbc Abuse Indicator':'abuse_type', 'Facility Type':'facility_type'}, inplace=True)

Drop Adult Foster Homes and select columns.

In [77]:
five_year_complaints = detailed[['abuse_number','facility_id','facility_type','incident_date','outcome_code',
                      'action_notes','outcome_notes','abuse_type']][detailed['facility_type']!='AFH']

No longer need the facility_type field.

In [78]:
five_year_ready = five_year_complaints.drop('facility_type',1)

There are thousands of complaints that appear in both datasets. If a complaint is a duplicate, we want to keep the one that is in the five-year set, because that one has richer data. To do this, we will add a 'source' column to each dataframe, value '1' for the five-year data and '2' for the ten-year data. We will then sort based on that column, then de-duplicate on the abuse_number field, telling pandas to keep the first instance of the duplicate that it finds.

In [79]:
five_year_ready['source']=1

In [80]:
ten_year_ready['source']=2

In [81]:
five_ten_concat = pd.concat([five_year_ready,ten_year_ready])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


Set abuse_numbers to uppercase (three abuse numbers in ten-year data have lowercase)

In [82]:
five_ten_concat['abuse_number'] = five_ten_concat['abuse_number'].apply(lambda x:x.upper())

In [83]:
five_ten_concat = five_ten_concat.sort_values('source')

In [84]:
complaints = five_ten_concat.drop_duplicates(subset='abuse_number', keep='first').reset_index().drop('index',1)

Add a 'year' column based on incident date.

In [85]:
complaints['year']=complaints['incident_date'].dt.year.astype(int)

In [86]:
complaints.count()

abuse_number     13705
abuse_type       12478
action_notes      6574
facility_id      13705
incident_date    13705
outcome_code     13704
outcome_notes     6544
source           13705
year             13705
dtype: int64

In [87]:
complaints['abuse_type'].fillna('',inplace=True)

<h3>Clean the abuse_type column</h3>

In [88]:
complaints['abuse_type'] = complaints['abuse_type'].apply(lambda x: x.upper())

In [89]:
complaints["abuse_type"] = complaints["abuse_type"].apply(dict([
    ('0', ''),  
    ('1', ''),  
    ('2', ''),  
    ('363', ''),  
    ('I', ''),
    ('A', 'A'),
    ('L', 'L'),
]).get).fillna('')

<h2>Join with scraped complaints</h2>

Complaints were scraped from https://apps.state.or.us/cf2/spd/facility_complaints/ using the script in `..scraper/DHS_scraper.py`

In [90]:
scraped_comp = pd.read_csv('../../data/scraped/scraped_complaints_3_25.csv')

Set all abuse numbers to upper case. 

In [91]:
scraped_comp['abuse_number'] = scraped_comp['abuse_number'].apply(lambda x: x.upper())

In [92]:
scraped_comp = scraped_comp.drop_duplicates(subset='abuse_number').drop(['fac_type','inv_comp_date','city_name'],1)

In [93]:
merged = complaints.merge(scraped_comp, how = 'left',on = 'abuse_number')

In [94]:
merged['outcome_code'] = merged['outcome_code'].fillna(0)

Add a column that tells us if the complaint has an equivalent online, based on the present of the online name.

In [95]:
merged['public'] = np.where(merged['fac_name'].notnull(),'online','offline')

In [96]:
# Added by Steve
merged.head()

,abuse_number,abuse_type,action_notes,facility_id,incident_date,outcome_code,outcome_notes,source,year,fac_name,online_incident_date,public
0,OT105179A,L,The RP failed to provide appropriate care for ...,385008,2010-08-31,500.0,RV reported asking staff to change him/her pri...,1,2010,NaN,NaN,offline
1,RB149539,A,The facility failed to update the care plan to...,5MA254,2014-12-07,534.0,The facility failed to assess and intervene in...,1,2014,EMERITUS AT MANOR HOUSE,12/07/2014,online
2,RB149241,,The facility failed to follow the care plan.,5MA254,2014-11-07,534.0,The facility allegedly failed to follow Reside...,1,2014,NaN,NaN,offline
3,RB145670,L,The facility failed to follow the care plan.,5MA254,2014-01-07,534.0,Reported Perpetrator 2 (RP2) failed to follow ...,1,2014,NaN,NaN,offline
4,RB135409,A,The facility failed to protect residents from ...,5MA254,2013-12-10,517.0,Resident #1 was care planned to be directed aw...,1,2013,EMERITUS AT MANOR HOUSE,12/10/2013,online


<h3>Join to a lookup table for the code number</h3>

In [97]:
codes = pd.read_excel('../../data/raw/OLRO Outcome Codes.xlsx', header=3)
codes.rename(columns = {'Code':'outcome_code','Display Text':'outcome'}, inplace = True)
codes['outcome_code'] = codes['outcome_code'].astype(str)
codes = codes.drop('Definition',1)

In [98]:
# Added by Steve, what's a lookup table? Is it a sort like a translation table? of tabular dictionary? 
codes.head()

,outcome_code,outcome
0,400,Failed to submit timely or adequate staffing d...
1,401,Failed to properly post and maintain daily sta...
2,500,Loss of Dignity
3,501,Denied Choice In Treatment
4,502,"Improperly Transferred Out of Facility, Denied..."


In [99]:
merged['outcome_code'] = merged['outcome_code'].astype(int).astype(str)

In [100]:
# Added by Steve
print('merged: ' + str(merged.shape[0]) + ' | codes: ' + str(codes.shape[0]))

merged: 13705 | codes: 52


In [101]:
merged = merged.merge(codes, how = 'left')

In [102]:
# added by Steve, so what just happened?
merged.head()

,abuse_number,abuse_type,action_notes,facility_id,incident_date,outcome_code,outcome_notes,source,year,fac_name,online_incident_date,public,outcome
0,OT105179A,L,The RP failed to provide appropriate care for ...,385008,2010-08-31,500,RV reported asking staff to change him/her pri...,1,2010,NaN,NaN,offline,Loss of Dignity
1,RB149539,A,The facility failed to update the care plan to...,5MA254,2014-12-07,534,The facility failed to assess and intervene in...,1,2014,EMERITUS AT MANOR HOUSE,12/07/2014,online,Fall with Injury
2,RB149241,,The facility failed to follow the care plan.,5MA254,2014-11-07,534,The facility allegedly failed to follow Reside...,1,2014,NaN,NaN,offline,Fall with Injury
3,RB145670,L,The facility failed to follow the care plan.,5MA254,2014-01-07,534,Reported Perpetrator 2 (RP2) failed to follow ...,1,2014,NaN,NaN,offline,Fall with Injury
4,RB135409,A,The facility failed to protect residents from ...,5MA254,2013-12-10,517,Resident #1 was care planned to be directed aw...,1,2013,EMERITUS AT MANOR HOUSE,12/10/2013,online,"Negative Behavior Escalated, Affected Other Re..."


In [103]:
merged.groupby('abuse_type').count()

,abuse_number,action_notes,facility_id,incident_date,outcome_code,outcome_notes,source,year,fac_name,online_incident_date,public,outcome
abuse_type,,,,,,,,,,,,
,1231,1219,1231,1231,1231,1166,1231,1231,426,426,1231,1230
A,3836,2101,3836,3836,3836,2120,3836,3836,3759,3759,3836,3836
L,8638,3254,8638,8638,8638,3258,8638,8638,1232,1232,8638,8636


In [104]:
merged['fac_name'].fillna('',inplace=True)

<h1>Join with facilities</h1>

<h4>First, prep the facilities.</h4>

In [105]:
facilities = pd.read_csv('../../data/raw/APD_FacilityRecords.csv')

In [106]:
facilities.rename(columns={'FACID':'facid','Facility ID':'facility_id','FAC_CCMUNumber':'fac_ccmunumber','FAC_Type':'facility_type',
                          'FAC_Capacity':'fac_capacity','Facility Name':'facility_name','Facility Address':'street',
                          'Other Service':'other_service','Owner':'owner','Operator':'operator'}, inplace=True)

Select the columns we need and drop the one duplicate in here. 

In [107]:
facilities = facilities[['facility_id','fac_ccmunumber','facility_type','fac_capacity','facility_name']].drop_duplicates(subset='facility_id', keep='last')

Churchill Estates Residential Care has blank facility_type and capacity fields. The facility is an RCF and has 108 capacity. Info obtained from DHS PIO.

In [108]:
facilities.loc[318,'facility_type']='RCF'
facilities.loc[318,'fac_capacity']=108

<h3>Left join facilities to complaints.</h3>

This eliminates complaints without facilities.

In [109]:
merged_comp_fac = facilities.merge(merged, on = 'facility_id',how = 'left')

The analysis is only of complaints in 2005 or later. 

In [110]:
merged_comp_fac = merged_comp_fac[['abuse_number','facility_id','facility_type','facility_name','abuse_type','action_notes','incident_date','outcome','outcome_notes',
                                   'year','fac_name','public']][merged_comp_fac['year']>2004]

In [111]:
# Added by Steve
merged_comp_fac.describe(include='all')

,abuse_number,facility_id,facility_type,facility_name,abuse_type,action_notes,incident_date,outcome,outcome_notes,year,fac_name,public
count,13032,13032,13032,13032,13032,6481,13032,13030,6450,13032.000000,13032,13032
unique,13032,599,3,599,3,3249,3727,49,6407,NaN,735,2
top,BH150570,5MA170,NF,Brookdale McMinnville Westside,L,The facility failed to provide a safe environm...,2014-11-14 00:00:00,Exposed to Potential Harm,Resident #1 had money go missing from his/her ...,NaN,,offline
freq,1,127,4760,127,8133,367,15,2317,15,NaN,7856,7856
first,NaN,NaN,NaN,NaN,NaN,NaN,2005-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-16 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.268416,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.198269,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,NaN


merged_comp_fac has all the complaints we need for the complaints analysis.

<h2>Aggregate data by facility</h2>

In [112]:
complaint_pivot = merged_comp_fac.pivot_table(values='abuse_number',index='facility_id',columns='public', aggfunc='count').reset_index()

In [113]:
# Added by Steve
complaint_pivot.head()

public,facility_id,offline,online
0,385008,14.0,7.0
1,385010,29.0,15.0
2,385015,27.0,8.0
3,385018,18.0,5.0
4,385024,67.0,27.0


Next, left join the facilities to the pivot table.

In [114]:
fac_pivot_merge = facilities.merge(complaint_pivot, how='left',on='facility_id')

In [115]:
# Added by Steve
fac_pivot_merge.head()

,facility_id,fac_ccmunumber,facility_type,fac_capacity,facility_name,offline,online
0,385008,385008,NF,96.0,Presbyterian Community Care Center,14.0,7.0
1,385010,385010,NF,159.0,Laurelhurst Village Rehabilitation Center,29.0,15.0
2,385015,385015,NF,128.0,Regency Gresham Nursing & Rehabilitation Center,27.0,8.0
3,385018,385018,NF,98.0,Providence Benedictine Nursing Center,18.0,5.0
4,385024,385024,NF,91.0,Avamere Health Services of Rogue Valley,67.0,27.0


<h2>Add our own outcome code</h2>

In [116]:
# Added by Steve
merged_comp_fac["outcome"].unique()

array(['Loss of Dignity', 'Medical Condition Developed or Worsened',
       'Burned', 'Fractured Bone', 'Exposed to Potential Harm',
       'Fall with Injury', 'Incontinence',
       'Received Incorrect or Wrong Dose of Medication(s)',
       'Fall Without Injury',
       'Transfer Resulting In Skin Injury or Bruise',
       'Mentally or Emotionally Abused',
       'The resident did not receive an ordered medication',
       'Fall Resulting In Fractured Bone(s)',
       'Improperly Transferred Out of Facility, Denied Readmission or Inappropriate Move Within Facility',
       'Bruised', 'Transfer To Hospital For Treatment',
       'Negative Behavior Escalated, Affected Other Resident(s)',
       'Injury During Self-Transfer', 'Decubitus Ulcer(s) Worsened',
       'Loss of Medication', 'Fall Resulting In Fractured Hip',
       'Skin Injury', 'Loss of Resident Property',
       'Unreasonable Discomfort', 'Decubitus Ulcer(s) Developed',
       'Pain And Suffering Continued', 'Financially E

In [117]:
# Added by Steve
dict([('No Negative Outcome', 'Potential harm')])

{'No Negative Outcome': 'Potential harm'}

In [118]:
merged_comp_fac["omg_outcome"] = merged_comp_fac["outcome"].apply(dict([
    ('No Negative Outcome', 'Potential harm'),
    ('Exposed to Potential Harm', 'Potential harm'),
            
    ('Fall Without Injury', 'Fall, no injury'),
            
    ('Left facility without assistance without injury', 'Left facility without attendant, no injury'),
            
    ('Loss of Dignity', 'Loss of Dignity'),
            
    ('Fall with Injury', 'Fracture or other injury'),
    ('Injury During Self-Transfer', 'Fracture or other injury'),
    ('Fall Resulting In Fractured Bone(s)', 'Fracture or other injury'),
    ('Fall Resulting In Fractured Hip', 'Fracture or other injury'),
    ('Transfer Resulting In Skin Injury or Bruise', 'Fracture or other injury'),
    ('Fractured Bone', 'Fracture or other injury'),
    ('Fractured Hip', 'Fracture or other injury'),
    ('Burned', 'Fracture or other injury'),
    ('Transfer Resulting In Fractured Hip', 'Fracture or other injury'),
    ('Transfer Resulting In Fracture Bone(s)', 'Fracture or other injury'),
    ('Left Facility Without Assistance With Injury', 'Fracture or other injury'),
    ('Bruised', 'Fracture or other injury'),
    ('Skin Injury', 'Fracture or other injury'),
            
    ('Negative Behavior Escalated, Affected Other Resident(s)', 'Failure to address resident aggression'),
            
    ('Medical Condition Developed or Worsened', 'Medical condition developed or worsened'),
    ('Decubitus Ulcer(s) Developed', 'Medical condition developed or worsened'),
    ('Decubitus Ulcer(s) Worsened', 'Medical condition developed or worsened'),
    ('Urinary Tract Infection Worsened', 'Medical condition developed or worsened'),
    ('Transfer To Hospital For Treatment', 'Medical condition developed or worsened'),
            
    ('Received Incorrect or Wrong Dose of Medication(s)', 'Medication error'),
    ('The resident did not receive an ordered medication', 'Medication error'),
            
    ('Loss of Resident Property', 'Loss of property, theft or financial exploitation'),
    ('Loss of Medication', 'Loss of property, theft or financial exploitation'),
    ('Financially Exploited', 'Loss of property, theft or financial exploitation'),
            
    ('Unreasonable Discomfort', 'Unreasonable discomfort or continued pain'),
    ('Pain And Suffering Continued', 'Unreasonable discomfort or continued pain'),
            
    ('Undesirable Weight Loss', 'Weight loss'),
            
    ('Poor Continuity Of Care', 'Inadequate care'),
    ('Failed To Have Quality of Life Maintained or Enhanced', 'Inadequate care'),
    ('Failed to Receive Needed Services', 'Inadequate care'),
    ('Denied Choice In Treatment', 'Inadequate care'),
            
    ('Incontinence', 'Inadequate hygiene'),
    ('Inadequate Hygiene', 'Inadequate hygiene'),
            
    ('Physically Abused', 'Physical abuse'),
    ('Corporally Punished', 'Physical abuse'),
            
    ('Verbally Abused', 'Verbal or emotional abuse'),
    ('Mentally or Emotionally Abused', 'Verbal or emotional abuse'),
            
    ('Involuntarily Secluded', 'Involuntary seclusion'),
            
    ('Raped', 'Sexual abuse'),
    ('Sexually Abused', 'Sexual abuse'),
            
    ('Deceased', 'Death'),
    ('Facility was understaffed with no negative outcome', 'Staffing issues'),
    ('Unable to timely assess adequacy of staffing', 'Staffing issues'),
            
    ('Improperly Transferred Out of Facility, Denied Readmission or Inappropriate Move Within Facility', 'Denied readmission or moved improperly'),
]).get).fillna('')

<h2>Export the facility and complaints data for munging</h2>

In [119]:
merged_comp_fac.to_csv('../../data/processed/complaints-3-25-scrape.csv',index=False)

In [120]:
fac_pivot_merge.to_csv('../../data/processed/facilities-3-25-scrape.csv',index=False)

<h1>DONE</h1>